# **Import Library**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer # Untuk menghitung TF-IDF
from sklearn import preprocessing # Untuk mengubah label menjadi angka
import pandas as pd # Untuk membaca data
from sklearn.model_selection import train_test_split # Untuk membagi data menjadi data latih dan data uji
import nltk # Untuk pemrosesan bahasa alami
from nltk.corpus import stopwords # Untuk menghapus kata-kata yang tidak penting
from nltk.stem import WordNetLemmatizer # Untuk mengubah kata ke bentuk dasar
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report # Untuk mengevaluasi model
import numpy as np # aljabar linear
import pandas as pd # pemrosesan data, CSV file I/O (misalnya pd.read_csv)
import string # untuk operasi string

lemma = WordNetLemmatizer() # Inisialisasi WordNetLemmatizer
# Download data untuk pemrosesan bahasa alami
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Menghubungkan ke google drive
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive

# **Load Datasets**

In [ ]:
data = pd.read_csv('news-article-categories.csv') # Membaca file CSV
df = pd.DataFrame(data) # Membuat DataFrame dari data
df.tail(10) # Menampilkan 10 data terakhir

# **Removing NULL Values**

In [ ]:
df.isnull().sum() # Digunakan untuk menghitung jumlah nilai null (NaN) dalam setiap kolom DataFrame df

In [ ]:
# Digunakan untuk menghapus baris dan kolom dari DataFrame df yang mengandung setidaknya satu nilai null (NaN).
df = df.dropna()
df = df.dropna(axis=1)

In [ ]:
df

# **Encoding Category**

In [ ]:
df['category'].value_counts() #  Untuk menghitung jumlah kemunculan setiap nilai unik dalam kolom "category" dari DataFrame df yang berguna untuk melihat distribusi kategori berita dalam data

In [ ]:
# Untuk mengonversi nilai kategori dalam kolom "category" menjadi angka
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(df['category'])
df['label'] = label_encoder.transform(df['category'])

In [ ]:
df

# **Text Preperation**

# **Lower Case**

In [ ]:
# Untuk mengubah semua teks dalam kolom "body" dari DataFrame df menjadi huruf kecil (lowercase)
df['body']=df['body'].str.lower()
df.head()

# **Remove HTML Tags**

In [ ]:
# Import modul BeautifulSoup
from bs4 import BeautifulSoup

# Mengecek apakah ada tag html didalam teks

# Mendefinisikan Fungsi 'has_html_tags'
def has_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    return bool(soup.find())

df['has_html_tags'] = df['body'].apply(has_html_tags)
df.head()

In [ ]:
count_true = df['has_html_tags'].sum()
count_true # jumlah total baris di mana tag HTML ditemukan dalam DataFrame

In [ ]:
df = df.drop(df[df['has_html_tags']].index) # mengembalikan baris-baris di mana tag HTML ditemukan dalam teks pada kolom "body".

In [ ]:
# Untuk menghapus kolom 'has_html_tags' dari DataFrame df
df = df.drop('has_html_tags', axis=1)
df

# **Remove Emojies**

In [ ]:
import regex # Mengimpor modul regex yang digunakan untuk bekerja dengan ekspresi reguler yang mendukung Unicode

# Mendefinisikan Fungsi 'has_emoji'
def has_emoji(text):
    emoji_pattern = regex.compile(r'\p{Emoji}', flags=regex.UNICODE)
    return bool(emoji_pattern.search(text))


has_emojis =  df['body'].apply(has_emoji)

has_emojis

In [ ]:
has_emojis.sum() #untuk menghitung jumlah nilai True dalam has_emojis

In [ ]:
# Mendefinisikan fungsi remove_emojis
def remove_emojis(text):
    emoji_pattern = regex.compile(r'\p{Emoji}', flags=regex.UNICODE)
    return emoji_pattern.sub('', text)

df['body'] = df['body'].apply(remove_emojis)

In [ ]:
# Untuk mengecek apakah setiap baris dalam kolom "body" yang sudah menggunakan metode apply dari DataFrame df mengandung emoji atau tidak
# (True jika emoji ditemukan, False jika tidak)
has_emojis =  df['body'].apply(has_emoji)
has_emojis

In [ ]:
has_emojis.sum() #untuk menghitung jumlah nilai True dalam has_emojis

# **Remove urls**

In [ ]:
import re # Mengimpor modul re yang digunakan untuk bekerja dengan ekspresi reguler (regular expressions) di Python.

# Mendefinisikan fungsi 'remove_url'
def remove_url(text):
    pattern=re.compile(r'https?://\S+|www\.S+')
    return pattern.sub(r'',text)
df["body"]=df["body"].apply(remove_url)

# **Tokenisasi, Remove punctuation, Remove stopwords**

In [ ]:
# Impor library
from string import punctuation

string.punctuation
def DataPrep(text) :
    # Tokenisasi
    tokens = nltk.word_tokenize(text)

    # Menghapus tanda baca
    punc = list(punctuation)
    words = [w for w in tokens if w not in punc]

    # Menghapus stopwords
    stop_words = set(stopwords.words('english'))

    # lemmatization
    words = [lemma.lemmatize(w) for w in words]

    text = ' '.join(words)

    return text

# **TF-IDF Vectorizer**

In [ ]:
x = df['body'] = df['body'].apply(DataPrep) # Menggunakan kolom "body" dari dataframe df dan menerapkannya ke fungsi DataPrep menggunakan metode apply.
y = df['label']  # Menggunakan kolom "label" sebagai target

# Vektorisasi teks menggunakan TF-IDF
vectorizer = TfidfVectorizer(stop_words = 'english', max_features=6000)
vectorized_x = vectorizer.fit_transform(x.values.astype('U'))

# **Split**

In [ ]:
# Asumsikan fitur vectorized_x sebagai fitur dan 'y' merupakan vektor label
# Gunakan stratify untuk memastikan bahwa pembagian data dilakukan dengan mempertahankan distribusi kelas yang sama seperti di dataset asli.
x_train, x_test, y_train, y_test = train_test_split(vectorized_x, y, test_size=0.1, stratify=y, random_state=0)

# **Smote**

In [ ]:
# Impor library
from imblearn.over_sampling import SMOTE

# Ganti Random Over Sampling dengan SMOTE
smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(x_train, y_train)

# Buat DataFrame baru
os = pd.DataFrame(list(zip([x[0] for x in X_resampled], y_resampled)), columns=['body', 'Label'])

# Tampilkan distribusi kelas setelah penerapan SMOTE
os['Label'].value_counts()

# **Model**

# **SVM**

In [ ]:
# Eka Pramuditya
# Mengimpor library yang dibutuhkan
from sklearn.svm import SVC

# Membuat model
classifier = SVC(random_state=42)

# Pelatihan model
classifier.fit(X_resampled, y_resampled)

# Prediksi pada data pengujian (x_test)
y_pred = classifier.predict(x_test)

# Evaluasi model
SVM_ACCURACY = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy_score(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
print(cm)

print(classification_report(y_test, y_pred))

# **Logistic Regression**

In [ ]:
# Angger Haryo Putranto
# Mengimpor library yang dibutuhkan
from sklearn.linear_model import LogisticRegression

# Membuat model
model = LogisticRegression(random_state=42, max_iter=200)

# Pelatihan model
model.fit(X_resampled, y_resampled)

# Prediksi pada data pengujian (x_test)
y_pred = model.predict(x_test)

# Evaluasi model
print("Accuracy:", accuracy_score(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print(cm)
print("Classification Report:")
print(classification_report(y_test, y_pred))

# **Stochastic Gradient Descent**

In [ ]:
# Wahid Hidayat

from sklearn.linear_model import SGDClassifier # Mengimpor SGDClassifier

classifier = SGDClassifier(random_state=0) # Menggunakan SGDClassifier dengan random_state=0

classifier.fit(X_resampled, y_resampled) # Melatih model dengan data latih yang sudah di-resampling

# Prediksi pada data pengujian (x_test)
y_pred = classifier.predict(x_test) # Melakukan prediksi pada data pengujian

# Evaluasi model
SGD_ACCURACY = accuracy_score(y_test, y_pred) # Menghitung akurasi model
print("Accuracy:", accuracy_score(y_test, y_pred)) # Menampilkan akurasi model

cm = confusion_matrix(y_test, y_pred) # Menghitung confusion matrix
print(cm) # Menampilkan confusion matrix

print("Classification Report:") # Menampilkan classification report
print(classification_report(y_test, y_pred)) # Menampilkan classification report dari model

#**Ensemble Stacking Classifier With SVM, Decision Tree, and logistic regression**

In [ ]:
# Farhan Husyen Ramadhan
# Mengimpor library yang dibutuhkan
from sklearn.ensemble import StackingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

# Membuat model
model1 = SVC(random_state=42)
model2 = DecisionTreeClassifier(random_state=0)
model3 = LogisticRegression(random_state=0)

# Membuat StackingClassifier
ensembleSC = StackingClassifier(estimators=[('svc', model1), ('dt', model2), ('lr', model3)], final_estimator=SVC())

# Pelatihan model
ensembleSC.fit(X_resampled, y_resampled)

# Evaluasi model pada data pelatihan
print(f"Akurasi training: {ensembleSC.score(x_train, y_train)}")

# Evaluasi model pada data pengujian
y_pred = ensembleSC.predict(x_test)
print(f"Akurasi prediksi: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))